In [2]:
import pickle, re, numpy as np, pandas as pd, math
from pprint import pprint

# mat_file = 'matrix5000.hdf'
mat_file = 'smatrix.hdf'
ppmi_file = 'ppmi20000.hdf'
svd_file = 'svd20000.hdf'

In [ ]:
def ppmi(mat_file):
    matrix = pd.read_hdf(mat_file,'mat1')
    
    cols = matrix.sum(axis=1)
    rows = matrix.sum(axis=0)
    total = cols.sum()
    print(cols.dtype)
    print(len(matrix.index), "total:", int(total))

    prod = np.multiply.outer(rows,cols) / total
    prod[1,1] == (rows[1]*cols[1] / total)
    
    matrix = matrix.divide(prod)
    matrix = matrix.apply(np.log2)
    matrix[matrix < 0] = 0

    matrix.to_hdf(ppmi_file,"mat1", mode='w')
ppmi(mat_file)

In [ ]:
def ppmi2svd(file):
    X = pd.read_hdf(file, 'mat1')
    print(X.iloc[0].sum())
    print(X.iloc[0])
    print(file + " read")
    
    U, s , V = np.linalg.svd(X, full_matrices=False)
    print("SVD done")
    
    U = pd.DataFrame(U, index= X.index, columns=X.columns)
    U.to_hdf(svd_file, 'mat1', mode='w')
    
    return U

U = ppmi2svd(ppmi_file)



In [ ]:
U

In [ ]:

matrix = pd.read_hdf(svd_file,'mat1')
matrix = matrix.iloc[:,:500]

In [3]:
matrix = pd.read_hdf(ppmi_file,'mat1')

In [7]:
# fly(1b) is to plane(1a) as drive(2b) is to
# ||wx − wdrive + wfly − wplane||2
# ||wx + (− an2a + an1b − an1a)||2
# ||wx + comb||2

def vecThing(an1a, an1b, an2b):
    v1a = matrix.loc[an1a].values
    v1b = matrix.loc[an1b].values
    v2b = matrix.loc[an2b].values
    
    comb = v1b - v1a - v2b
#     print("v1a", v1a)
#     print()
#     print("v1b", v1b)    
#     print()
#     print("v2a", v2a)
#     print()
#     print("comb", comb)
#     print()
#     print("mat", matrix.values)
#     print()
#     print("mat + comb",matrix.values + comb)
#     print()
#     print(np.linalg.norm(matrix.iloc[1].values + comb))
    
    dists = np.linalg.norm(matrix.values + comb, axis=1) ** 2
    return dists

In [8]:
# np.linalg.norm(np.arange(1, 4) - np.arange(0,3)) ** 2
# np.arange(1, 4) - np.arange(0,3)
matrix.iloc[2].sum()
# matrix

298.29593

In [12]:

an1b = 'bird'
an1a = 'sky'
an2b = 'fish'

dists = vecThing(an1a, an1b, an2b)

dists = pd.Series(dists, index=matrix.index).sort_values()

In [13]:
print(np.mean(dists),min(dists),max(dists))

32703.32421875 16715.1 52938.0


In [15]:
print(dists['water'])
print(dists[:20])


43714.5
sky              16715.148438
fish             19163.099609
han'             28552.816406
harvie           28680.380859
turbulence       28714.041016
newt             28768.861328
homesick         28872.566406
ominously        28877.675781
saleslady        28888.416016
refill           28904.388672
busts            28912.804688
stomachs         28915.093750
marmara          28919.759766
laminate         28927.222656
silhouetted      28929.636719
neared           28932.906250
satisfies        28946.570312
gloriana         28963.707031
subtracting      28968.142578
109-degrees-f    28989.162109
dtype: float32
